# Лабораторная работа по заданию №  8. Практический анализ современных алгоритмов

## Раздел 4 - Разделяй и властвуй. 4.1. Задача поиска максимального подмассива

In [39]:
import matplotlib.pyplot as plt

In [154]:
def find_max_crossing_subarray(A, low, mid, high):
    print(low, mid, high)
    left_sum = float("-inf")
    summ = 0
    for i in reversed(range(low, mid+1)):  
        print(i)
        summ = summ+A[i]
        print('sum:'+str(summ))
        if summ > left_sum:
            left_sum = summ
            max_left = i
    right_sum = float("-inf")
    summ=0
    print(mid+1, high)
    for j in range(mid+1, high):
        print(j)
        summ=summ+A[j]
        print('sum:'+str(summ))
        if summ>right_sum:
            right_sum=summ
            max_right=j
    return (max_left, max_right, left_sum+right_sum)

def find_maximum_subarray(A, low, high):
    print(low, high)
    if high==low:
        print('high==low')
        print(low)
        return (low, high, A[int(low)])
    else:  
        print('high!=low')
        mid=(low+high)//2
        left_low, left_high, left_sum = find_maximum_subarray(A, low, mid)
        right_low, right_high, right_sum = find_maximum_subarray(A, mid+1, high)
        cross_low, cross_high, cross_sum = find_max_crossing_subarray(A, low, mid, high)
        if (left_sum>=right_sum) and (left_sum>=cross_sum):
            return (left_low, left_high, left_sum)
        elif (right_sum>=left_sum) and (right_sum>=cross_sum):
            return right_low, right_high, right_sum
        else:
            return cross_low, cross_high, cross_sum

In [153]:
A=[13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7]
print(find_maximum_subarray(A, 1, len(A)))
#print(find_max_crossing_subarray(A, 0, (len(A)+1)//2, len(A)))

1 16
high!=low
1 8
high!=low
1 4
high!=low
1 2
high!=low
1 1
high==low
1
2 2
high==low
2
1 1 2
1
sum:13
2 2


UnboundLocalError: local variable 'max_right' referenced before assignment